In [1]:
import pandas as pd

In [2]:
# converte o arquivo xlsx para csv, para otimizar a consulta no Pandas
ano = 2017
file_name = f'OD_{ano}.xlsx'
df = pd.read_excel(file_name)
file_name_csv = file_name.replace('.xlsx', '.csv')
df.to_csv(file_name_csv, index=False)
print('Arquivo csv salvo com sucesso.')

# converte o arquivo xlsx para csv, para otimizar a consulta no Pandas
df = pd.read_csv(file_name_csv, low_memory=False)
df.info()

Arquivo csv salvo com sucesso.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183092 entries, 0 to 183091
Columns: 188 entries, ZONA - Zona do Domicílio to Município 3ª Transferência
dtypes: float64(95), int64(33), object(60)
memory usage: 262.6+ MB


In [5]:
file_name = 'projecoes_2018_populacao_2010_2060_20200406.xls'
# seleciona a Aba "SP"
df_ibge = pd.read_excel(file_name, sheet_name='SP', skiprows=50, skipfooter=220)

# remove as golunas desnecessárias (2010 até 2016)
for coluna_a_remover in range(2010, 2017):
    del df_ibge[coluna_a_remover]

len(df_ibge)
print(df_ibge.columns)
df_ibge.tail()

Index(['GRUPO ETÁRIO',           2017,           2018,           2019,
                 2020,           2021,           2022,           2023,
                 2024,           2025,           2026,           2027,
                 2028,           2029,           2030,           2031,
                 2032,           2033,           2034,           2035,
                 2036,           2037,           2038,           2039,
                 2040,           2041,           2042,           2043,
                 2044,           2045,           2046,           2047,
                 2048,           2049,           2050,           2051,
                 2052,           2053,           2054,           2055,
                 2056,           2057,           2058,           2059,
                 2060],
      dtype='object')


,GRUPO ETÁRIO,2017,2018,2019,2020,2021,2022,2023,2024,2025,...,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060
15,70-74,1105574,1168609,1233506,1297671,1361395,1424192,1486762,1550543,1616172,...,2888999,2945598,2996231,3036590,3064120,3079097,3081698,3073036,3055027,3029789
16,75-79,766399,787462,814670,850465,893859,944807,1000959,1058711,1115864,...,2262396,2318031,2377989,2437532,2493191,2545804,2596099,2641044,2676754,2700969
17,80-84,499510,521489,542741,562367,580946,598292,616948,640551,670980,...,1707970,1729242,1751699,1779126,1813521,1854036,1900486,1950273,1999579,2045600
18,85-89,277317,286565,297067,309125,322854,338660,355440,371651,386673,...,1151917,1173253,1193472,1212535,1230636,1247974,1264305,1281618,1302713,1328991
19,90+,166426,180113,193341,205952,218341,230838,243602,256905,270793,...,1008214,1046169,1083184,1118975,1153388,1186590,1218796,1249863,1279611,1308095


In [6]:
df_faixas_etarias = df_ibge['GRUPO ETÁRIO']
# converte a Series em dataframe
df_faixas_etarias = df_faixas_etarias.to_frame()
# remove última linha, que é NA
df_faixas_etarias = df_faixas_etarias.dropna()
# Remove a primeira linha, que é "Total"
df_faixas_etarias = df_faixas_etarias[1:]
df_faixas_etarias

,GRUPO ETÁRIO
1,0-4
2,5-9
3,10-14
4,15-19
5,20-24
6,25-29
7,30-34
8,35-39
9,40-44
10,45-49


In [ ]:
# cria as colunas que serão usadas para classificar as pessoas dentro do range
df_faixas_etarias[['idade_minima', 'idade_maxima']] = df_faixas_etarias['GRUPO ETÁRIO'].str.split('-', expand = True)
# ajusta a idade_maxima dos 90+
df_faixas_etarias['idade_maxima'].fillna(150, inplace=True)
# remove o sinal de + para a idade de 90 anos
df_faixas_etarias['idade_minima'] = df_faixas_etarias['idade_minima'].str.replace('+', '', regex=False)
df_faixas_etarias.rename(columns={'GRUPO ETÁRIO': 'grupo_etario'}, inplace=True)
# altera os campos para numéricos
df_faixas_etarias['idade_minima'] = pd.to_numeric(df_faixas_etarias['idade_minima'])
df_faixas_etarias['idade_maxima'] = pd.to_numeric(df_faixas_etarias['idade_maxima'])
df_faixas_etarias

In [ ]:
# a partir dos grupos etários da lista do ibge, cria a coluna 
# grupo_etario no dataframe OD2017, incluindo o seu grupo etário
grupos = [
    0,
    4,
    9,
    14,
    19,
    24,
    29,
    34,
    39,
    44,
    49,
    54,
    59,
    64,
    69,
    74,
    79,
    84,
    89,
    150
]
labels = df_faixas_etarias['grupo_etario'].to_list()
df['grupo_etario'] = pd.cut(
    df['IDADE - Idade'],
    bins=grupos,
    include_lowest=False,
    precision=0,
    labels=labels
)

df[['IDADE - Idade', 'grupo_etario']].tail()
df[['ID_PESS - Identifica Pessoa', 'IDADE - Idade', 'grupo_etario']].head()

In [ ]:
# cria uma cópia apenas dos campos que serão usados para fazer o 'JOIN' 
# com o dataframe que já foi lido no Power BI
df_saida = df[['ID_PESS - Identifica Pessoa', 'IDADE - Idade', 'grupo_etario']].copy()
a_renomear = {
    'ID_PESS - Identifica Pessoa': 'ID_PESS',
    'IDADE - Idade': 'IDADE',
    'grupo_etario': 'GRUPO ETÁRIO'
}
df_saida.rename(columns=a_renomear, inplace=True)
df_saida.head(10)

In [ ]:
# faz o merge dos dados do IBGE com o df_saida
df_saida_ibge = df_saida.merge(df_ibge)
# remove registros duplicados, se houver
print('Antes de remover duplicados ->', len(df_saida_ibge))
df_saida_ibge.drop_duplicates(inplace=True)
print('Após remover duplicados ->', len(df_saida_ibge))
df_saida_ibge.head()

In [ ]:
df_saida_ibge.head()

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
# cria algumas colunas com o fator de evolução, no formato
# FATOR_ANO_FINAL_ANO_INICIAL
ano_inicial, ano_final = 2017, 2023
df_saida_ibge[f'FATOR_{ano_inicial}_{ano_final}'] = (df_saida_ibge[ano_final]/df_saida_ibge[ano_inicial])

ano_inicial, ano_final = 2017, 2025
df_saida_ibge[f'FATOR_{ano_inicial}_{ano_final}'] = (df_saida_ibge[ano_final]/df_saida_ibge[ano_inicial])

ano_inicial, ano_final = 2017, 2035
df_saida_ibge[f'FATOR_{ano_inicial}_{ano_final}'] = (df_saida_ibge[ano_final]/df_saida_ibge[ano_inicial])

ano_inicial, ano_final = 2017, 2040
df_saida_ibge[f'FATOR_{ano_inicial}_{ano_final}'] = (df_saida_ibge[ano_final]/df_saida_ibge[ano_inicial])

ano_inicial, ano_final = 2017, 2045
df_saida_ibge[f'FATOR_{ano_inicial}_{ano_final}'] = (df_saida_ibge[ano_final]/df_saida_ibge[ano_inicial])

ano_inicial, ano_final = 2017, 2050
df_saida_ibge[f'FATOR_{ano_inicial}_{ano_final}'] = (df_saida_ibge[ano_final]/df_saida_ibge[ano_inicial])

ano_inicial, ano_final = 2017, 2055
df_saida_ibge[f'FATOR_{ano_inicial}_{ano_final}'] = (df_saida_ibge[ano_final]/df_saida_ibge[ano_inicial])

ano_inicial, ano_final = 2017, 2060
df_saida_ibge[f'FATOR_{ano_inicial}_{ano_final}'] = (df_saida_ibge[ano_final]/df_saida_ibge[ano_inicial])

df_saida_ibge.head()

In [ ]:
df_saida_ibge.columns

In [ ]:
file_path_saida = 'OD_2017_fator_de_comparação_IBGE.xlsx'
df_saida_ibge.to_excel(file_path_saida, index=False)
print('Arquivo de saída salvo com sucesso.')